In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:

from projimmo.data import hello
hello()

hello toi


In [2]:
import sys
sys.path

['/home/cpoulard/code/CecilePoulard/04-Decision-Science/01-Project-Setup/data-context-and-setup',
 '/home/cpoulard/code/CecilePoulard/projimmo',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/cpoulard/.pyenv/versions/3.10.6/envs/projimmo/lib/python3.10/site-packages']

In [3]:
import projimmo

projimmo.__file__

'/home/cpoulard/code/CecilePoulard/projimmo/projimmo/__init__.py'

In [2]:
import os
import pandas as pd
from projimmo.params import *
#data=os.getenv('DATA_DIR')
print(DATA_DIR)

chunk = pd.read_csv(DATA_DIR, sep="|", nrows=CHUNK_SIZE, dtype=str)
chunk.head()
#'/home/cpoulard/projimmo/raw_data/valeursfoncieres-2023.txt'


raw_data/valeursfoncieres-2023.txt


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,2,Appartement,NaN,233,8,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,03/01/2023,Vente,...,NaN,0,1,Maison,NaN,64,3,S,NaN,988
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,0,1,Maison,NaN,73,3,S,NaN,835


In [1]:
#Ecrire dans la bdd avec BigQuery

import numpy as np
import pandas as pd

from google.cloud import bigquery
from pathlib import Path
from colorama import Fore, Style


from projimmo.params import *

TABLE='test'
table=f"{GCP_PROJECT}.{BQ_DATASET}.{TABLE}"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

client = bigquery.Client()

write_mode = "WRITE_TRUNCATE" # or "WRITE_APPEND"
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

job = client.load_table_from_dataframe(df, table, job_config=job_config)
result = job.result()

In [2]:

import pandas as pd
import random
import seaborn as sns
from scipy import stats
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler,OrdinalEncoder
import pandas as pd
from pathlib import Path
from projimmo.data import *
from projimmo.preprocessor import *

query = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.DVF_{DATA_YEAR}
    """

    # $CHA_BEGIN
    # Retrieve data using `get_data_with_cache`
#data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
 #   data_query = get_data_with_cache(
 #       query=query,
 #       gcp_project=GCP_PROJECT,
 #       cache_path=data_query_cache_path,
 #       data_has_header=True
 #   ).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
data_query_cache_path=Path(f"raw_data/valeursfoncieres-{DATA_YEAR}.txt")
# Lire le fichier entier
#df = pd.read_csv("raw_data/valeursfoncieres-2023.txt", sep="|", dtype=str)
print(data_query_cache_path)
df= get_data_with_cache(query=query,
gcp_project=GCP_PROJECT,
        cache_path=data_query_cache_path,
        data_has_header=True
   )


#load_data_to_bq(
#        data_processed_with_timestamp,
 #       gcp_project=GCP_PROJECT,
 #       bq_dataset=BQ_DATASET,
 #       table=f'processed_{DATA_SIZE}',
 #       truncate=True
  #  )

df_0=clean_data(df)

df.dtypes
df_1=clean_outliers(df_0)
#load_data_to_bq(
#        df,
 #       gcp_project=GCP_PROJECT,
#        bq_dataset=BQ_DATASET,
 #       table=f'clean_data_{DATA_YEAR}',
 #       truncate=True
 #   )

df_2=preprocess_features(df_1)


raw_data/valeursfoncieres-2023.txt

Load data from local CSV...
✅ Data loaded, with shape (3727005, 43)


/home/cpoulard/code/CecilePoulard/projimmo/projimmo/data.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[carrez_cols].fillna(0,inplace=True)
/home/cpoulard/code/CecilePoulard/projimmo/projimmo/data.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Surface reelle bati'


✅ X_processed, with shape (75666, 768)


In [10]:
df_2.head(20)

,surface_reelle_bati,surface_terrain,somme_surface_carrez,valeur_fonciere,code_type_local_1,code_type_local_2,type_de_voie_ACH,type_de_voie_ALL,type_de_voie_ART,type_de_voie_AV,...,departement_75015,departement_75016,departement_75017,departement_75018,departement_75019,departement_75020,nombre_pieces_principales,month_mutation,year_mutation,3
0,1.312723,NaN,1.215123,0.071429,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0
1,1.312723,NaN,1.135831,0.142857,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0
2,-0.374117,NaN,-0.169891,-0.607143,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
3,0.064462,NaN,0.196049,-0.380952,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
4,0.300619,NaN,0.302316,-0.357143,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0
5,1.245249,NaN,1.528474,-0.285714,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
6,0.165672,NaN,0.384877,-0.642857,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
7,1.481407,NaN,1.841008,0.071429,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0
8,0.806671,NaN,0.641553,-0.595238,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0
9,-0.272906,NaN,-0.383515,-0.490476,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0


In [2]:
df.head(20)

,valeur_fonciere,type_de_voie,departement,nombre_de_lots,code_type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,month_mutation,year_mutation,somme_surface_carrez
654,230000.0,RUE,1330,1,2,95.0,4.0,NaN,1,2023,97.70
1027,245000.0,RUE,1330,2,2,95.0,4.0,NaN,1,2023,94.79
1598,87500.0,RUE,1340,1,2,45.0,2.0,NaN,1,2023,46.87
2101,135000.0,nan,1310,1,2,58.0,3.0,NaN,1,2023,60.30
2112,140000.0,RUE,1330,1,2,65.0,3.0,NaN,2,2023,64.20
2268,155000.0,RUE,1360,1,2,93.0,3.0,NaN,1,2023,109.20
2564,80000.0,RUE,1370,1,2,61.0,2.0,NaN,1,2023,67.23
2781,230000.0,RTE,1350,1,2,100.0,5.0,NaN,1,2023,120.67
2998,90000.0,RUE,1340,1,2,80.0,4.0,NaN,1,2023,76.65
3374,112000.0,RTE,1360,1,2,48.0,3.0,NaN,1,2023,39.03
